# BIOB480/BIOE548 Final Project

To conclude the semester, students will work independently to analyze empirical genetic or genomic data from a population of wild plants or animals. Specifically, each of you will select a published dataset and population genetic metric, read the abstract of the associated scientific article, load the data into a Jupyter Notebook, and perform simple analyses using the Python package scikit-allel. Learning outcomes include gaining familiarity with Python and scriptable anlayses of genomic data more broadly, learning how sequence data and genotypes are encoded into data objects. The project is worth 25 points and must be submitted as an `.ipynb` notebook in your local directory on Tempest. Of these points, 10 are for completeness, while 15 are earned by answers to the questions.

## Setup

As before, we're going to load several `Python` libraries to facilitate analysis of genotype data. 

In [2]:
import os
import allel
import numpy as np
import pandas as pd
np.set_printoptions(legacy='1.25')

## Selecting a Dataset

To start, visit the [linked GitHub repository](https://github.com/elinck/conservation_genetics) and read the README.md file on the main page. There, I introduce a set of empirical conservation genetics datasets, with links to compressed Variant Call Format (`.vcf`) files and their associated papers. I've already moved these data to the subdirectory `/biob-480-genetics/course-materials/data/` on Tempest, so all you need to do is decide on study taxon of interest. Once you've done so, read at least the abstract of the linked paper, and review all figures and tables. This will inform your answer to **QUESTION 1** below.

**QUESTION 1**: Create a new code chunk immediately below this field. Click inside the box and select `Markdown` from the dropdown field that currently reads `Code` above. Markdown is a simple formatting language for text——surrounding words with a pair of asterices (e.g., `*text*`) makes them *italic*, while two pairs of asterices (`**text**`) makes them **bold**. Clicking the "run" button will then format all text in the code chunk according to `Markdown`'s syntax (a handy cheatsheet of other rules [is available here](https://www.markdownguide.org/cheat-sheet/). Though the formatting is not important, you'll use `Markdown` chunks whenever you need to respond to a question with written text. Here, I'd like you to state which dataset you selected, and briefly summarize the question, genetic market type, and main findings in your own words. Why did it interest you?

## Loading and examining variant data

Next up, we're going to pivot to looking at empirical data: specifically, the SNP data associated with your project. These data are in Variant Call Format, and can be found as `*.vcf.gz` in the tutorial folder. In the example below, we'll load a test dataset present in the `course-materials/` subdirectory. 

In [3]:
file = "/home/group/biob-480-genetics/course-materials/data/iris.vcf.gz"
s = allel.read_vcf(file, fields='*')

The numpy array object `s` has a set of keys representing aspects of the `.vcf` file. We can list these with the `.keys()` function, sorting by name:

In [1]:
sorted(s.keys())

NameError: name 's' is not defined

Here, anything begining with "callset" is from the INFO field of the .vcf, while "variants" refers to data associated with each SNP (or site, if you have invariant alleles in your file). (Some of this information, e.g. depth, is redundant, depending on your SNP caller.) To examine any of these in greater detail, we can select the relevant index. For example, to look at the position of these SNPs (relative to the reference genome or sequence they were aligned to), we can select `variants/POS` key. Here, we use brackets with a `String` inside to select a column with a particular label:

In [ ]:
s['variants/POS']

Similarly, we can list all sample IDs, which in most cases will be filenames from one of the various bioinformatics processing steps used to generate the final `.vcf` file:

In [ ]:
s['samples']

**QUESTION 2**: How many samples are in your dataset? To answer this, you may add a new code chunk below and comment directly within it next to your script with a hashtag (`#`), or add a `Markdown` chunk above your code chunk to add written text (e.g., "There are 102 samples in my dataset:", followed by a separate line with the code you used to determine your answer).

Besides simply looking at these data, these keys can be used to filter SNPs based on various criteria. First, though, you may want to know how many SNPs you acutally have. We can evaluate this by taking the first entry from the shape attribute of the array (i.e., its length): 

In [ ]:
s['calldata/GT'].shape[0]

Next, let's take the actual genotype calls from the vcf-turned-numpy-array and turn it into the GenotypeArray object we're familiar with:

In [ ]:
gt = s['calldata/GT']
gt = allel.GenotypeArray(gt)
gt

**QUESTION 3**: How many loci are in your dataset?

## Metrics of Genetic Diversity

Let's calculate the frequencies of different alleles (SNPs) using the `count_alleles().to_frequencies()` function introduced above, saving them to a new object.

In [3]:
af_gt = gt.count_alleles().to_frequencies()
af_gt

NameError: name 'gt' is not defined

We can select a single site using brackets—for example, the following line of code gives us the frequencies of the two alleles at the second site in the genome (recall that `Python` uses 0-indexing, i.e. the first site is `[0]`:

In [ ]:
af_gt[1]

We can use the following function to calculate observed heterozygosity at each SNP in the dataset:

In [ ]:
h_o = allel.heterozygosity_observed(gt)
h_o

**QUESTION 4**: What is the observed heterozygosity at the 32nd site in your data?

The function allel.heterozygosity_expected works similarly:

In [ ]:
h_e = allel.heterozygosity_expected(af_gt, ploidy=2)
h_e

Recall that $F = 1 - \frac{H_o}{H_e}$. We therefore have enough to calculate the value of the inbreeding coefficient for any locus:

In [ ]:
 allel.inbreeding_coefficient(gt)

**QUESTION 5**: Calculate $H_e$, $H_o$, and $F$ for SNPs at positions 6, 19, and 72. 

Another basic descriptive task in conservation and population genetics is assessing the diversity across loci in a set of samples. Our empiricial data have already been filtered down to variant sites, so we'll work with the simple genotype array `g` from last lesson to get a hang of how a few functions work. To begin, we need to load it again:  

In [ ]:
g = allel.GenotypeArray([[[0, 0], [0, 0]],
                         [[0, 0], [0, 1]],
                         [[0, 0], [1, 1]],
                         [[0, 1], [1, 1]],
                         [[1, 1], [1, 1]],
                         [[0, 0], [1, 1]],
                         [[0, 1], [1, 0]],
                         [[0, 1], [-1, -1]],
                         [[-1, -1], [-1, -1]]])

We next need to convert our genotype array into counts of each allele at each position. (Note that the maximum value will be 2N=4.)

In [ ]:
g_ac = g.count_alleles()
g_ac

There are 9 sites in this allele count array. Let's imagine they correspond to positions 2,4,7,14,15,18,19, and 27 in a reference genome that is 32 nucleotides long. We can indicate this with an array assigned to `pos`, and then calculate $\pi$ using the function `allel.sequence_diversity()`. (Note that positions 1 through 31 indicate the stretch of the genome these SNPs are taken from, including invariant sites.)

In [5]:
pos = [2, 4, 7, 14, 15, 18, 19, 25, 27]
pi = allel.sequence_diversity(pos, g_ac, start=1, stop=31)
pi

NameError: name 'allel' is not defined

Another useful metric of diversity is the mean number of pairwise differences for a particular locus. For SNP data, this is equivalent to comparing each allele in each individual and scoring them as 0 (no difference) or 1 (different alleles). We can calculate this for all SNPs in our empirical data

In [ ]:
gt_ac = gt.count_alleles()
dif = allel.mean_pairwise_difference(gt_ac)
dif

**QUESTION 6**: Remember that basic Python 3 function `max()` calculates the maximum value of a given array. What is the largest mean pairwise difference value in your dataset?

## Linkage Disequilibrium 

Recall that linkage disequilibrium—the association of alleles at different loci at great—is typically measured by the metric $D$. A related metric, $r$, can be interpreted as the [correlation between two diploid genotypes](doi.org/10.1534/genetics.108.093153). We can calculate this for our empirical data after first manipulating our genotype array to instead display the number of non-reference alleles (i.e., we are coding diploid genotypes for each individual at each SNP by the number of alleles they have that are different than the reference genome):

In [ ]:
gt_subset = gt[1:50]
gt_an = gt_subset.to_n_alt(fill=-1)
gt_an[41]

A seperate function returns a matrix of pairwise (sample to sample) LD values:

In [ ]:
ld = allel.rogers_huff_r(gt_an)
ld

We can plot these using a built in function:

In [ ]:
allel.plot_pairwise_ld(ld, colorbar=True, ax=None, imshow_kwargs=None)

**QUESTION 7**: What is the largest value of $r$ in your dataset? Is it low or high?

Let's plot a histogram of these same data. What do you notice?

In [8]:
plt.hist(ld, bins='auto') # arguments are passed to np.histogram
plt.title("Histogram of pairwise $r$ values")
plt.show()

NameError: name 'plt' is not defined

## Population Subdivision

Most (all?) of your datasets include samples from multiple, geographically isolated populations. Properly analyzing these data would require linking sample IDs to specific populations, which should be possible, but is difficult to standardize across datasets for our purposes. Therefore, you can decide to choose one of two options to estimate $F_{ST}$ or your dataset: 1) an arbitrary assignment of samples to "subpopulations"; or 2) accurately assigning individuals to subpopulations on the basis of the source paper and / or other files in the [Dryad repository linked on GitHub](https://github.com/elinck/conservation_genetics).

For the first option, we will arbitrarily divide our samples and associated alelle count arrays into two groups:

In [ ]:
first_half = int(gt.n_samples/2) # select a sample number half-way through the total number of samples
second_half = int(gt.n_samples) # select the final sample
subpops = list(range(first_half)), list(range(first_half+1,second_half)) # assign subpopulations as a list of sample numbers
gt_ac1 = gt.count_alleles(subpop=subpops[0]) # calculate allele counts for the first "subpopulation"
gt_ac2 = gt.count_alleles(subpop=subpops[1]) # calculate allele counts for the second "subpopulation"
num, den = allel.hudson_fst(gt_ac1, gt_ac2) # calculate the numerator and denominator for FST for each SNP
fst = np.sum(num) / np.sum(den) # calculate average FST by summing over all SNPs
fst

**QUESTION 8**: What is the genome-wide average $F_{ST}$ value for your data calculated in this way? Why is it so low?

Recall that we loaded our `.vcf` files as an object called `s`. Among its attributes is a list of samples—let's display the first 5 sample names as an array:

In [ ]:
s['samples'][0:5]

By inspecting this file and comparing it to background information, we can theoretically assign individuals to actual subpopulations by their index. If possible, modify the following code to do so, calculating $F_{ST}$ between two of separate populations in your study. (If there are no lists of sample IDs / localities available, you may select another random subset.)

In [ ]:
subpops2 = [[0,1,2,3,4],[90,91,92,93,94]] # this would assign individuals 0-4 and 90-94 to two different subpopulations; edit as needed
subpops2
gt_ac1_v2 = gt.count_alleles(subpop=subpops2[0]) 
gt_ac2_v2 = gt.count_alleles(subpop=subpops2[1]) 
num, den = allel.hudson_fst(gt_ac1_v2, gt_ac2_v2) 
fst = np.nansum(num) / np.nansum(den) 
fst

**QUESTION 9**: Is this $F_{ST}$ value lower or higher than your answer to **QUESTION 8**?

Let's also calculate *per variant* $F_{ST}$---this way, we can see whether certain SNPs / loci are outliers (and thus potentially under selection). 

In [10]:
np.seterr(divide='ignore', invalid='ignore') # allow division by zero
fst = num / den 
fst = fst[~np.isnan(fst)] # drop nan values

NameError: name 'np' is not defined

As with our LD values, we can plot a histogram of per-SNP $F_{ST}$: 

In [ ]:
plt.hist(fst, bins='auto')
plt.title("Histogram of per-SNP $F_{ST}$ values")
plt.show()

**QUESTION 10**: What is the maximum $F_{ST}$ value in your dataset? Be precise (i.e., use a function and don't just eyeball the plot). For an extra challenge, determine its position in your dataset.

## Principal Component Analysis 

To conclude, we'll perform Principal Component Analysis (PCA) on our data. Though we did not explicitly cover PCA in class, it is a ubiquitous approach to extracting patterns from highly multivariate datsets (like genotypes from thousands of SNPs and hundreds of samples) that involves placing samples on new, "synthetic" axes that explain the majority of variation in the data. A brief introduction to PCA [is here](https://en.wikipedia.org/wiki/Principal_component_analysis); arguably its most famous use in population genetics is [this 2008 paper in Nature](https://www.nature.com/articles/nature07331) by John Novembre and colleagues. PCA is most useful when we know the identity of each plotted sample, but even without going through the necessary steps to do so with our own data, the patterns it reveals can be informative.

`scikit-allel` has a built-in PCA function, which is straightforward to implement once we are sure our data are complete (i.e., have no missing values). To do so, we'll start by counting alleles as before:

In [ ]:
ac_sub = gt.count_alleles() # count alleles
ac_sub

Because some of the datasets include invariant sites, which will cause problems with PCA, we need to drop them from the object. To do so, we will combine `Numpy`'s `np.nonzero` function with the `is_variant` method of `allel`'s genotype datastructures. (The "0" below makes sure we end up with a single array; don't worry about it.)

In [ ]:
indices = np.nonzero(ac_sub.is_variant())[0]

Once we have these values, we can use the `take` method of genotype datastructures to subset them:

In [ ]:
indices = np.nonzero(ac_sub.is_variant())[0] # identify only variant sites
genotypes_pca = gt.take(indices, axis=0) # subset genotypes to variants

In [ ]:
before = len(gt)
after = len(genotypes_pca)
SNPs_dropped = before - after
SNPs_dropped

In [ ]:
(Depending on your dataset, this value may be zero.)

Our last data preparation step is to transform our data to an array counts of alternate alleles—i.e., alleles that are different than the reference sequence reads were aligned to during sequence assembly:

In [12]:
gn = genotypes_pca.to_n_alt()
gn

NameError: name 'genotypes_pca' is not defined

Now it's time to run PCA itself. Note that we are saving information on the model used to run it in the second variable.

In [ ]:
coords, model = allel.pca(gn) # run pca

In [ ]:
plt.scatter(coords[:, 0], # position on the first principal component of the observations
            coords[:, 1], alpha=0.7);

**QUESTION 11**: Interpret this PCA. Do you see distinct clusters, suggesting groups of samples that are closely related to each other, or do you see largely continuous variation? What biological processes relevant to your study taxon might help explain these patterns? Do your results match conclusions in the paper?

That's the end of your brief tour of `scikit-allel`! We have, of course, only scratched the surface of what you can do with sequence data in Python. If you are motivated, feel free to add new code chunks below and perform an addititional analysis or data visualization on your dataset using any resource you wish for help. If your work is successful and you document it, I'll give you extra credit (**QUESTION 12**)!

**QUESTION 13**: Please fill out an evaluation for this course using [this link (for undegraduates)](https://montana.campuslabs.com/eval-home/direct/8519695) or [this link (for graduate students)](https://montana.campuslabs.com/eval-home/direct/6550795). Confirm you have done so in a new `Markdown` chunk below.

 

**QUESTION 14**: Did you enjoy the programming activities in this class? What could be done to improve them for future semesters?

**QUESTION 15:** What was most surprising, exciting, or disappointing to you about this course?